In [3]:
import fitz  
import cv2
import numpy as np
from PIL import Image
import io


In [4]:

def sharpen_image_from_pixmap(pix):
    # Convert pixmap to bytes
    img_bytes = pix.tobytes("ppm")  # Get image bytes in PPM format
    img_array = np.frombuffer(img_bytes, dtype=np.uint8)

    # Decode with OpenCV
    img = cv2.imdecode(img_array, cv2.IMREAD_COLOR)

    # Apply sharpening kernel
    sharpen_kernel = np.array([[0, -1, 0],
                               [-1, 5, -1],
                               [0, -1, 0]])
    sharpened_img = cv2.filter2D(img, -1, sharpen_kernel)

    # Convert back to bytes using PIL
    img_pil = Image.fromarray(cv2.cvtColor(sharpened_img, cv2.COLOR_BGR2RGB))
    buf = io.BytesIO()
    img_pil.save(buf, format="PNG")
    buf.seek(0)

    # Create new pixmap from bytes
    new_pix = fitz.Pixmap(buf)
    return new_pix


In [5]:
input_path = r"C:\Users\santh\Downloads\Telegram Desktop\Santhosh_Kumar_MCA.pdf"
outputpath = r"C:\Users\santh\Downloads\Telegram Desktop\Santhosh_Kumar_MCA1.pdf"

In [14]:
import fitz  # PyMuPDF
import numpy as np
import cv2
from PIL import Image

def convert2_non_editable_pdfs(input_path, output_path):
    try:
        doc = fitz.open(input_path)
        new_doc = fitz.open()

        for i in range(len(doc)):
            page = doc[i]
            mat = fitz.Matrix(2, 2)  # increase resolution
            pix = page.get_pixmap(matrix=mat, alpha=False)

            # Convert pix to NumPy array
            img = np.frombuffer(pix.samples, dtype=np.uint8).reshape(pix.height, pix.width, pix.n)

            # Apply sharpening filter
            kernel = np.array([[0,-1,0],[-1,5,-1],[0,-1,0]])
            sharpened_img = cv2.filter2D(img, -1, kernel)

            # Convert back to Pixmap using PIL
            img_pil = Image.fromarray(sharpened_img)
            img_rgb = img_pil.convert("RGB")
            img_bytes = img_rgb.tobytes()
            pixmap = fitz.Pixmap(fitz.csRGB, (img_rgb.width, img_rgb.height), img_bytes)

            # Add the sharpened image to a new page
            new_page = new_doc.new_page(width=pixmap.width, height=pixmap.height)
            new_page.insert_image(fitz.Rect(0, 0, pixmap.width, pixmap.height), pixmap=pixmap)

        new_doc.save(output_path)
        print("PDF has been converted to non-editable")
        return True

    except Exception as e:
        print("Exception raised in pdf conversion: " + str(e))
        return False


In [15]:
convert2_non_editable_pdf(input_path,outputpath)

PDF has been converted to non-editable with sharpened pages.


True